# Data Collection and Pre‑Processing Lab : E‑commerce (12‑Step Roadmap)

**Author:** Oluwafemi Lawal
**Date:** 2025-09-30

This notebook executes the 12‑step Data Engineering roadmap on a realistic e‑commerce–style dataset.  
It loads raw data, cleans and enriches it, and finishes with a concise analytical insight.  
A **Data Dictionary** merged from a secondary metadata source follows at the end.


## Setup

In [ ]:

import os
import math
from dataclasses import dataclass
from typing import Dict, List, Any, Optional

import pandas as pd
import numpy as np

pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

PRIMARY_CSV_CANDIDATES = [
    "data/100000 Sales Records.csv",
    "../data/100000 Sales Records.csv",
]
primary_csv = next((p for p in PRIMARY_CSV_CANDIDATES if os.path.exists(p)), None)
if primary_csv is None:
    raise FileNotFoundError("Could not find '100000 Sales Records.csv' in data/.")

print("Using primary CSV:", primary_csv)


Using primary CSV: data/100000 Sales Records.csv


## Step 1 : Hello, Data!
Load raw CSV and display the first 3 rows (I keep only the first 500 rows for the assignment).


In [2]:

raw_df = pd.read_csv(primary_csv)
raw_df = raw_df.head(500).copy()
raw_df.head(3)


,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Middle East and North Africa,Azerbaijan,Snacks,Online,C,10/8/2014,535113847,10/23/2014,934,152.58,97.44,142509.72,91008.96,51500.76
1,Central America and the Caribbean,Panama,Cosmetics,Offline,L,2/22/2015,874708545,2/27/2015,4551,437.20,263.33,1989697.20,1198414.83,791282.37
2,Sub-Saharan Africa,Sao Tome and Principe,Fruits,Offline,M,12/9/2015,854349935,1/18/2016,9986,9.33,6.92,93169.38,69103.12,24066.26


## Step 2 : Pick the Right Container
**Justification (dict vs namedtuple vs set):**  
- **dict** is ideal for keyed lookups (e.g., `city -> revenue`) and flexible schemas while I iterate.  
- **namedtuple/dataclass** suits *row-like* records with fixed fields for clarity and type hints.  
- **set** is perfect for uniqueness checks (e.g., distinct cities).  
For this workflow, I’ll use **dicts** for aggregations and a **dataclass** to model a cleaned order record.


## Step 3 : Implement Functions and Data Structure
I’ll synthesize required fields not present in the raw file:
- `date` ← `Order Date`
- `customer_id` ← derived from `Order ID`
- `product` ← `Item Type`
- `price` ← `Unit Price`
- `quantity` ← `Units Sold`
- `coupon_code` ← heuristic (based on `Order Priority` & `Item Type`)
- `shipping_city` ← heuristic mapping from `Country` (major/capital city)

I provide a small `OrderRecord` dataclass and a `Cleaner` class with `.clean()` and `.total()`.


In [ ]:

from dataclasses import dataclass
from datetime import datetime

# Simple secondary metadata (will be persisted to CSV later for Data Dictionary merge)
# 1) Coupon catalog with human-readable descriptions & percent discounts
COUPON_META = pd.DataFrame([
    {"coupon_code":"NOCOUP", "description":"No promotion applied", "discount_pct":0},
    {"coupon_code":"SAVE10", "description":"10% off standard items", "discount_pct":10},
    {"coupon_code":"SAVE15", "description":"15% off select items", "discount_pct":15},
    {"coupon_code":"VIP20",  "description":"20% VIP customer promo", "discount_pct":20},
], dtype="object")

# 2) Country -> representative shipping city (approximate; illustrative for the lab)
COUNTRY_CITY_META = pd.DataFrame([
    {"Country":"United States of America", "shipping_city":"New York"},
    {"Country":"Canada", "shipping_city":"Toronto"},
    {"Country":"Mexico", "shipping_city":"Mexico City"},
    {"Country":"Brazil", "shipping_city":"São Paulo"},
    {"Country":"Argentina", "shipping_city":"Buenos Aires"},
    {"Country":"United Kingdom", "shipping_city":"London"},
    {"Country":"France", "shipping_city":"Paris"},
    {"Country":"Germany", "shipping_city":"Berlin"},
    {"Country":"India", "shipping_city":"Mumbai"},
    {"Country":"China", "shipping_city":"Shanghai"},
], dtype="object")

# Heuristic: choose coupon from Order Priority & Item Type
def choose_coupon(order_priority: str, item_type: str) -> str:
    p = (order_priority or "").strip().upper()
    it = (item_type or "").strip().loIr()
    if p in {"H","C"} and it in {"cosmetics","household","snacks"}:
        return "SAVE15"
    if p in {"M"} or it in {"clothes","baby food"}:
        return "SAVE10"
    if it in {"personal care"}:
        return "VIP20"
    return "NOCOUP"

@dataclass
class OrderRecord:
    date: datetime
    customer_id: str
    product: str
    price: float
    quantity: int
    coupon_code: str
    shipping_city: str

class Cleaner:
    def __init__(self, df: pd.DataFrame):
        self.df = df.copy()

    def _coerce_types(self, df: pd.DataFrame) -> pd.DataFrame:
        # Parse dates
        for col in ["Order Date", "Ship Date"]:
            if col in df.columns:
                df[col] = pd.to_datetime(df[col], errors="coerce")
        # Numeric coercions
        for col in ["Units Sold","Unit Price","Unit Cost","Total Revenue","Total Cost","Total Profit"]:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors="coerce")
        return df

    def _add_required_fields(self, df: pd.DataFrame) -> pd.DataFrame:
        df["date"] = df["Order Date"]
        # derive deterministic pseudo-customer ids from Order ID
        df["customer_id"] = df["Order ID"].astype(str)
        df["product"] = df["Item Type"]
        df["price"] = df["Unit Price"]
        df["quantity"] = df["Units Sold"]
        # coupon heuristic
        df["coupon_code"] = [
            choose_coupon(op, it) for op, it in zip(df["Order Priority"], df["Item Type"])
        ]
        # map Country -> shipping_city; fallback: use Country name
        city_map = dict(zip(COUNTRY_CITY_META["Country"], COUNTRY_CITY_META["shipping_city"]))
        df["shipping_city"] = df["Country"].map(city_map).fillna(df["Country"])
        return df

    def clean(self) -> pd.DataFrame:
        """Apply cleaning rules and return cleaned DataFrame with required columns."""
        df = self.df.copy()
        before = len(df)
        df = self._coerce_types(df)

        # Drop rows with missing key fields
        df = df.dropna(subset=["Order Date","Order ID","Item Type","Units Sold","Unit Price","Country","Order Priority"])

        # Remove negative or zero quantities/prices (if any)
        df = df[(df["Units Sold"] > 0) & (df["Unit Price"] > 0)]

        # Add required fields
        df = self._add_required_fields(df)

        # Standardize text cases
        df["product"] = df["product"].astype(str).str.title()
        df["coupon_code"] = df["coupon_code"].astype(str).str.upper()
        df["shipping_city"] = df["shipping_city"].astype(str).str.title()

        after = len(df)
        print(f"Cleaned rows: before={before}, after={after}, removed={before-after}")
        return df

    def total(self, price: float, quantity: int, discount_pct: float) -> float:
        return float(price) * int(quantity) * (1 - float(discount_pct)/100.0)


## Step 4 : Bulk Loaded
Map from DataFrame rows to dictionaries to illustrate container choices.


In [ ]:

cleaner = Cleaner(raw_df)
clean_df = cleaner.clean()

# Join discount metadata so I can compute totals
clean_df = clean_df.merge(COUPON_META[["coupon_code","discount_pct"]], on="coupon_code", how="left")

# Map to a list of dictionaries (bulk loaded)
records: List[Dict[str, Any]] = []
for _, r in clean_df.iterrows():
    rec = {
        "date": r["date"],
        "customer_id": r["customer_id"],
        "product": r["product"],
        "price": float(r["price"]),
        "quantity": int(r["quantity"]),
        "coupon_code": r["coupon_code"],
        "discount_pct": float(r.get("discount_pct", 0) or 0.0),
        "shipping_city": r["shipping_city"],
    }
    records.append(rec)

len(records), records[0]


Cleaned rows: before=500, after=500, removed=0


(500,
 {'date': Timestamp('2014-10-08 00:00:00'),
  'customer_id': '535113847',
  'product': 'Snacks',
  'price': 152.58,
  'quantity': 934,
  'coupon_code': 'SAVE15',
  'discount_pct': 15.0,
  'shipping_city': 'Azerbaijan'})

## Step 5 : Quick Profiling
Compute min/mean/max price and the number of unique shipping cities (using a set).


In [5]:

price_min = clean_df["price"].min()
price_mean = clean_df["price"].mean()
price_max = clean_df["price"].max()
unique_cities = len(set(clean_df["shipping_city"].dropna().astype(str)))

print({"price_min": price_min, "price_mean": price_mean, "price_max": price_max, "unique_cities": unique_cities})


{'price_min': np.float64(9.33), 'price_mean': np.float64(278.65074), 'price_max': np.float64(668.27), 'unique_cities': 173}


## Step 6 : Spot the Grime
At least three examples of dirty data identified:
1. Non-parsable dates (`Order Date` or `Ship Date`) → coerced to `NaT` and dropped.
2. Missing/blank `Item Type`, `Units Sold`, or `Unit Price` → dropped.
3. Negative/zero `Units Sold` or `Unit Price` → dropped.


## Step 7 : Cleaning Rules
Fixes are executed in `Cleaner.clean()`. "Before/after" counts are printed during cleaning.


## Step 8 : Transformations
Parse `coupon_code` → numeric discount and compute per-row `net_total` using `.total()`.


In [6]:

clean_df["net_total"] = [
    cleaner.total(p, q, d) for p, q, d in zip(clean_df["price"], clean_df["quantity"], clean_df["discount_pct"].fillna(0))
]
clean_df[["customer_id","product","price","quantity","coupon_code","discount_pct","net_total"]].head(5)


C:\Users\femil\AppData\Local\Temp\ipykernel_30588\201086444.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaner.total(p, q, d) for p, q, d in zip(clean_df["price"], clean_df["quantity"], clean_df["discount_pct"].fillna(0))


,customer_id,product,price,quantity,coupon_code,discount_pct,net_total
0,535113847,Snacks,152.58,934,SAVE15,15,121133.262
1,874708545,Cosmetics,437.20,4551,NOCOUP,0,1989697.200
2,854349935,Fruits,9.33,9986,SAVE10,10,83852.442
3,892836844,Personal Care,81.73,9118,SAVE10,10,670692.726
4,129280602,Household,668.27,5858,SAVE15,15,3327516.811


## Step 9 : Feature Engineering
Example: `days_since_purchase` from the `date` column.


In [7]:

today = pd.Timestamp.today().normalize()
clean_df["days_since_purchase"] = (today - pd.to_datetime(clean_df["date"])).dt.days
clean_df[["date","days_since_purchase"]].head(5)


,date,days_since_purchase
0,2014-10-08,4009
1,2015-02-22,3872
2,2015-12-09,3582
3,2014-09-17,4030
4,2010-02-04,5716


## Step 10 : Mini‑Aggregation
Revenue per `shipping_city` using a dictionary and a pandas check.


In [8]:

# Dict accumulation
city_rev: Dict[str, float] = {}
for city, total in zip(clean_df["shipping_city"], clean_df["net_total"]):
    city_rev[city] = city_rev.get(city, 0.0) + float(total)

# Pandas check
city_rev_df = clean_df.groupby("shipping_city", as_index=False)["net_total"].sum().rename(columns={"net_total":"revenue"})
city_rev, city_rev_df.sort_values("revenue", ascending=False).head(5)


({'Azerbaijan': 1407375.662,
  'Panama': 11205616.317,
  'Sao Tome And Principe': 754545.1680000001,
  'Belize': 14874286.709499998,
  'Denmark': 3101908.7199999997,
  'Berlin': 3526354.3499999996,
  'Turkey': 5227259.8719999995,
  'London': 5762163.961000001,
  'Kazakhstan': 1927274.2339999997,
  'Haiti': 398656.00999999995,
  'Italy': 6185146.973999999,
  'Malta': 4169008.175,
  'Jordan': 6525311.34,
  'Cambodia': 2808989.6580000003,
  'Saint Kitts And Nevis ': 8134019.040000001,
  'Cameroon': 1367529.5999999999,
  'Bahrain': 363738.5,
  'Solomon Islands': 174995.6,
  'Monaco': 7264356.113,
  'Comoros': 1111031.0840000003,
  'Iceland': 1438373.484,
  'Zambia': 1535358.96,
  'Egypt': 5252491.301,
  'Togo': 810617.184,
  'Saudi Arabia': 7372561.662999999,
  'Morocco': 1534792.902,
  'Tunisia ': 7814874.592,
  'Angola': 5258437.1245,
  'Vietnam': 1827750.65,
  'Belarus': 3654833.07,
  'Myanmar': 1521298.8900000001,
  'Lithuania': 892614.8335,
  'Switzerland': 3590630.6180000002,
  'Anti

## Step 11 : Serialization Checkpoint
Save cleaned data to **JSON** (and CSV for convenience).


In [9]:

OUTPUT_DIR = "data"
os.makedirs(OUTPUT_DIR, exist_ok=True)

json_path = os.path.join(OUTPUT_DIR, "cleaned_transactions_ol.json")
csv_path  = os.path.join(OUTPUT_DIR, "cleaned_transactions_ol.csv")
meta_coupon_path = os.path.join(OUTPUT_DIR, "coupon_metadata_ol.csv")
meta_city_path = os.path.join(OUTPUT_DIR, "country_city_metadata_ol.csv")

# Persist primary outputs
clean_df.to_json(json_path, orient="records", lines=True)
clean_df.to_csv(csv_path, index=False)

# Persist secondary metadata used for the Data Dictionary merge
COUPON_META.to_csv(meta_coupon_path, index=False)
COUNTRY_CITY_META.to_csv(meta_city_path, index=False)

json_path, csv_path, meta_coupon_path, meta_city_path


('data\\cleaned_transactions_ol.json',
 'data\\cleaned_transactions_ol.csv',
 'data\\coupon_metadata_ol.csv',
 'data\\country_city_metadata_ol.csv')

## Step 12 : Soft Interview Reflection
Functions (and small classes) clarify intent, make tests possible, and reduce duplication.  
A dedicated `Cleaner` centralized parsing, validation, and standardization, which keeps downstream
code focused on analysis rather than edge cases. Encapsulating the pricing logic in `.total()`
made discount handling explicit and reusable across aggregations. As a result, the pipeline is
easier to extend: for example, adding shipping fees or tax rules only requires a small change in
one location. This modularity is what turns a quick prototype into a maintainable data workflow.


## Data‑Dictionary (Merged Two‑Source)
Below I merge field definitions from the primary CSV header and the secondary metadata sources
(coupon catalog and country→city lookup). New engineered columns are documented with their provenance.


In [10]:

# Build a tidy Data Dictionary
primary_fields = [
    ("Order Date","datetime64[ns]","Original order date","primary CSV"),
    ("Order ID","string/int","Unique order identifier","primary CSV"),
    ("Item Type","string","Product category label","primary CSV"),
    ("Units Sold","int","Units sold for the order line","primary CSV"),
    ("Unit Price","float","Unit price for the item type","primary CSV"),
    ("Country","string","Destination country","primary CSV"),
    ("Order Priority","string","Internal priority flag (L/M/H/C)","primary CSV"),
]

engineered_fields = [
    ("date","datetime64[ns]","Alias of Order Date","derived"),
    ("customer_id","string","Surrogate customer id (from Order ID)","derived"),
    ("product","string","Alias of Item Type (title‑cased)","derived"),
    ("price","float","Alias of Unit Price","derived"),
    ("quantity","int","Alias of Units Sold","derived"),
    ("coupon_code","string","Heuristic coupon assignment from priority/type","derived + coupon metadata"),
    ("discount_pct","float","Percent discount from coupon metadata","secondary: coupon catalog"),
    ("shipping_city","string","Representative city from country lookup","secondary: country→city"),
    ("net_total","float","price*quantity*(1-discount)","derived"),
    ("days_since_purchase","int","Days since `date`","derived"),
]

dd_df = pd.DataFrame(primary_fields + engineered_fields, columns=["Field","Type","Description","Source"])
dd_df


,Field,Type,Description,Source
0,Order Date,datetime64[ns],Original order date,primary CSV
1,Order ID,string/int,Unique order identifier,primary CSV
2,Item Type,string,Product category label,primary CSV
3,Units Sold,int,Units sold for the order line,primary CSV
4,Unit Price,float,Unit price for the item type,primary CSV
5,Country,string,Destination country,primary CSV
6,Order Priority,string,Internal priority flag (L/M/H/C),primary CSV
7,date,datetime64[ns],Alias of Order Date,derived
8,customer_id,string,Surrogate customer id (from Order ID),derived
9,product,string,Alias of Item Type (title‑cased),derived


## Mini Insight (One‑liner)
Top cities by net revenue suggest where logistics performance matters most; targeted promotions
in those regions could yield measurable gains.


In [11]:

city_rev_df.sort_values("revenue", ascending=False).head(10)


,shipping_city,revenue
51,Georgia,1.549567e+07
14,Belize,1.487429e+07
72,Kosovo,1.325734e+07
31,Croatia,1.122648e+07
114,Panama,1.120562e+07
22,Burundi,1.069189e+07
99,Mozambique,1.054990e+07
165,Uganda,1.014286e+07
15,Benin,9.771828e+06
155,The Bahamas,9.668468e+06
